In [1]:
import cv2
import numpy as np
import pykinect_azure as pykinect
import open3d as o3d

# Depth camera parameters:
FX_DEPTH = 503.988861
FY_DEPTH = 503.946228
CX_DEPTH = 320.462463
CY_DEPTH = 329.527344


def Compute_point_cloud(im):
    # compute point cloud:
    pcd = []
    height, width = im.shape
    # compute indices:
    jj = np.tile(range(width), height)
    ii = np.repeat(range(height), width)
    # Compute constants:
    xx = (jj - CX_DEPTH) / FX_DEPTH
    yy = (ii - CY_DEPTH) / FY_DEPTH
    # transform depth image to vector of z:
    length = height * width
    z = im.reshape(height * width)
    # compute point cloud
    pcd = np.dstack((xx * z, -yy * z, z)).reshape((length, 3))
    return pcd

def Compute_3D_blob_coord(keys,im,depth):
    key_3D = []
    depth_image_meters = depth / 1
    for blob in keys:
       j = round(blob.pt[0])
       i = round(blob.pt[1])
       z = depth_image_meters[j][i]
       x = (j - CX_DEPTH) * z / FX_DEPTH
       y = (i - CY_DEPTH) * z / FY_DEPTH
       key_3D.append([x, -y, z])
    

    pcd = o3d.geometry.PointCloud()  # create point cloud object
    pcd.points = o3d.utility.Vector3dVector(key_3D)  # set pcd_np as the point cloud points   
    vis2.clear_geometries()
    vis2.add_geometry(pcd)
    vis2.update_geometry(pcd)
    vis2.poll_events()
    vis2.update_renderer()   
    return key_3D

def Blob_detector(im,depth,detector):
    
    # Detect blobs.
    im_conv = (im/256).astype('uint8')
    depth_conv = (depth/256).astype('uint8')
    
    print(depth.shape)
    print(im_conv.shape)
    #cv2.imshow("Keypoints", im)
    ret,im_conv_treshold = cv2.threshold(im_conv,250,255,cv2.THRESH_BINARY)
    cv2.imshow("Tresh", im_conv_treshold)
    keypoints = detector.detect(im_conv_treshold)
    
    #cv2.imshow("Keypoints", im_conv)
    
    
    # Draw detected blobs as red circles.
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
    im_with_keypoints = cv2.drawKeypoints(im_conv_treshold, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    depth_with_keypoints = cv2.drawKeypoints(depth_conv, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    #for blob in keypoints:
        #cv2.circle(im_with_keypoints, (int(blob.pt[0]),int(blob.pt[1])), 1, (0,255,0), -1)

    cv2.imshow("Keypoints", im_with_keypoints)
    cv2.imshow("Keypoints_depth", depth_with_keypoints)


    print(Compute_3D_blob_coord(keypoints,im_conv,depth))
    threeDpoints = Compute_point_cloud(im_conv)
    pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
    pcd_o3d.points = o3d.utility.Vector3dVector(threeDpoints)  # set pcd_np as the point cloud points
    
    
    # Clear and update the Open3D visualizer
    
    vis.clear_geometries()
    vis.add_geometry(pcd_o3d)
    vis.update_geometry(pcd_o3d)
    vis.poll_events()
    vis.update_renderer()
    # Visualize:
    #o3d.visualization.draw_geometries([pcd_o3d])
    # Show keypoints
    #cv2.imshow("tracked", im_with_keypoints)
    



if __name__ == "__main__":

    # Initialize the library, if the library is not found, add the library path as argument
    pykinect.initialize_libraries()

    # Modify camera configuration
    device_config = pykinect.default_configuration
    #device_config.color_format = pykinect.K4A_COLOR_FORMAT_COLOR_BGRA32
    device_config.color_resolution = pykinect.K4A_COLOR_RESOLUTION_1440P
    device_config.depth_mode = pykinect.K4A_DEPTH_MODE_NFOV_UNBINNED
    print(device_config)

    # Initialize Open3D Visualizer
    #vis = o3d.visualization.Visualizer()
    
    vis = o3d.visualization.VisualizerWithKeyCallback()
    vis.create_window('viewer', 640, 576)

    vis2 = o3d.visualization.Visualizer()
    vis2.create_window('viewer2', 640, 576)
    # Start device
    device = pykinect.start_device(config=device_config)
    
    #cv2.namedWindow('Infrared Image',cv2.WINDOW_NORMAL)
    #cv2.namedWindow('tracked',cv2.WINDOW_NORMAL)
    
    # Set up the SimpleBlobDetector with default parameters
    params = cv2.SimpleBlobDetector_Params()

    # Set the threshold
    params.minThreshold = 100
    params.maxThreshold = 300

    # Set the area filter
    params.filterByArea = True
    params.minArea = 10
    params.maxArea = 1000

    # Set the circularity filter
    params.filterByCircularity = True
    params.minCircularity = 0.8
    params.maxCircularity = 1

    # Set the convexity filter
    params.filterByConvexity = False
    params.minConvexity = 0.2
    params.maxConvexity = 1

    # Set the inertia filter
    params.filterByInertia = False
    params.minInertiaRatio = 0.01
    params.maxInertiaRatio = 1
    
    # Set the color filter
    params.filterByColor = True
    params.blobColor = 255
    

    # Create a detector with the parameters
    detector = cv2.SimpleBlobDetector_create(params)
    
    
    while True:
   
        # Get capture
        capture = device.update()

        # Get the infrared image
        ret, ir_image = capture.get_ir_image()
        ret2, depth_image = capture.get_depth_image()
        
        if not ret:
            continue
        if not ret2:
            continue

        Blob_detector(ir_image,depth_image,detector)
        # Plot imageq
        #cv2.imshow('Infrared Image',ir_image)

        # Press q key to stop
        if cv2.waitKey(1) == ord('q'):  
            break

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Device configuration: 
	color_format: 0 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 3 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 2 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 us. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 us.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


(576, 640)
(

KeyboardInterrupt: 